In [ ]:
import pickle
from pathlib import Path

import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import configuration as config
from cardiac_electrophysiology.ulac import construction, workflow

In [ ]:
patient_id = "07"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = {"MV": 0, "LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [ ]:
ulac_construction_settings = construction.ULACConstructorSettings(
    mesh=mesh,
    feature_tags=anatomical_tags,
    path_config=config.path_configs,
    marker_config=config.marker_configs,
    submesh_config=config.submesh_configs,
    segmentation_workflow=workflow.workflow,
)
ulac_constructor = construction.ULACConstructor(ulac_construction_settings)
ulac_constructor.construct_segmentation()
ulac_constructor.construct_uacs()

In [ ]:
boundary_inds = ulac_constructor.data.submesh_boundary_data["lateral_mv"].inds
boundary_alpha = ulac_constructor.data.submesh_boundary_data["lateral_mv"].alpha

plotter = pv.Plotter()
plotter.add_mesh(mesh, opacity=0.3, show_edges=True)
plotter.add_mesh(mesh.points[boundary_inds], scalars=boundary_alpha, point_size=10, render_points_as_spheres=True)
plotter.show()

In [ ]:
uac_submesh_data = ulac_constructor.data.uac_submesh_data
uac_submesh_key_sequences = list(construction.nested_dict_keys(uac_submesh_data))

plotter = pv.Plotter()
for key_sequence in uac_submesh_key_sequences:
    uac_submesh = construction.get_dict_entry(key_sequence, uac_submesh_data)
    coordinates = np.hstack(
        (
            uac_submesh.alpha[:, None],
            uac_submesh.beta[:, None],
            np.zeros((uac_submesh.alpha.shape[0], 1))
        )
    )
    submesh = pv.PolyData.from_regular_faces(
        coordinates, uac_submesh.connectivity
    )
    plotter.add_mesh(submesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()

In [ ]:
marker_data = ulac_constructor.data.marker_data
uac_path_data = ulac_constructor.data.uac_path_data
marker_key_sequences = list(construction.nested_dict_keys(marker_data))
path_key_sequences = list(construction.nested_dict_keys(uac_path_data))

plotter = pv.Plotter(shape=(1, 2), window_size=[1000, 500])
for i, uac in enumerate(["alpha", "beta"]):
    plotter.subplot(0, i)
    plotter.add_mesh(mesh, style="wireframe", color="grey")

    for key_sequence in marker_key_sequences:
        marker = construction.get_dict_entry(key_sequence, marker_data)
        if marker is None:
            continue
        marker_mesh = pv.PolyData(mesh.points[marker.ind])
        plotter.add_mesh(marker_mesh, color="red", point_size=10, render_points_as_spheres=True)
    for key_sequence in path_key_sequences:
        path = construction.get_dict_entry(key_sequence, uac_path_data)
        if path is None:
            continue
        path_mesh = pv.PolyData(mesh.points[path.inds])
        path_mesh.point_data[uac] = getattr(path, uac)
        plotter.add_mesh(path_mesh, point_size=5, scalars=uac, render_points_as_spheres=True)

plotter.show()